In [1]:
# PDF Reader
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader('kpis.pdf')
pdf_documents = loader.load()


In [2]:
# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300
)

# Split the documents into chunks
documents = text_splitter.split_documents(pdf_documents)


In [ ]:
# Vector Embedding and Vector Store
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")  

# Create a vector store from documents
db = Chroma.from_documents(
    documents, 
    embedding_model, 
    persist_directory="./chroma_db"
)

In [6]:
db

In [7]:
# Query the database
query = "Why is this indicator important?"

response = db.similarity_search(query)

# Clean the result content (remove unwanted tab characters and extra spaces)
cleaned_response = response[0].page_content

print(cleaned_response)


actually  completed  at any given  point  in time.  
In addition  to assessing  progress  to date,  EV allows
 companies  to project  what the  likely  costs  of the complete
 project  will be, assuming  that performance  levels remain
 as they  have  been  to date.  
How  do I measure  it?


In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_7040\1732197747.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


Ollama()

## Retriever And Chain With Langchain

In [9]:

## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}""")

In [10]:
# Chain instruction 
# Create Stuff Documnt Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001810E5E8110>, search_kwargs={})

In [12]:
# Retriever Chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001810E5E8110>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \n<

In [13]:
response = retrieval_chain.invoke({"input":"Why is this indicator important?"})

In [14]:
response['answer']

"The indicator of carbon footprint is important for several reasons:\n\n1. Regulatory compliance: With increasing legislation and regulations around carbon emissions, measuring and managing a company's carbon footprint is essential to avoid legal penalties and reputational damage.\n2. Reputation and brand image: In today's socially conscious society, companies are under pressure to reduce their environmental impact. Measuring and reporting on carbon footprint can help companies demonstrate their commitment to sustainability and improve their reputation.\n3. Cost savings: Reducing energy consumption and carbon emissions can lead to significant cost savings over time. By measuring and managing a company's carbon footprint, businesses can identify areas for improvement and make strategic decisions to reduce their environmental impact.\n4. Long-term planning: Accurate measurement of a company's carbon footprint allows for informed long-term planning and decision-making related to energy co